Let's talk about how we can express models in PyTorch

In [1]:
import torch                     # for all things PyTorch
import torch.nn as nn            # for torch.nn.Module, the parent object for PyTorch models
import torch.nn.functional as F  # for the activation function

<img src="lenet5.png">

*Figura: LeNet-5*

Arriba hay un diagrama de LeNet-5, una de las primeras redes neuronales convolucionales, y uno de los impulsores de la explosión del aprendizaje profundo. Fue construida para leer pequeñas imágenes de números escritos a mano (el conjunto de datos MNIST), y clasificar correctamente qué dígito estaba representado en la imagen.

Esta es la versión abreviada de cómo funciona:

* La capa C1 es una capa convolucional, lo que significa que explora la imagen de entrada en busca de las características que ha aprendido durante el entrenamiento. Produce un mapa de dónde ha visto cada una de las características aprendidas en la imagen. Este "mapa de activación" se reduce en la capa S2.
* La capa C3 es otra capa convolucional, esta vez escanea el mapa de activación de C1 en busca de *combinaciones* de características. También emite un mapa de activación que describe las ubicaciones espaciales de estas combinaciones de características, que se reduce en la capa S4.
* Por último, las capas totalmente conectadas del final, F5, F6 y OUTPUT, son un *clasificador* que toma el mapa de activación final y lo clasifica en uno de los diez segmentos que representan los 10 dígitos.

¿Cómo expresamos esta sencilla red neuronal en código?


In [4]:
class LeNet(nn.Module):

    def __init__(self):
        super(LeNet, self).__init__()
        # 1 input image channel (black & white), 6 output channels, 3x3 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 3)
        self.conv2 = nn.Conv2d(6, 16, 3)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 6 * 6, 120)  # 6*6 from image dimension
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        print("MaxPool conv1",x)
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        print("MaxPool conv2",x)
        x = x.view(-1, self.num_flat_features(x))
        print("view num_flat_features",x)
        x = F.relu(self.fc1(x))
        print("relu fc1",x)
        x = F.relu(self.fc2(x))
        print("relu fc2",x)
        x = self.fc3(x)
        print("relu fc3",x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

Mirando este código, deberías ser capaz de ver algunas similitudes estructurales con el diagrama de arriba.

Esto demuestra la estructura de un modelo típico de PyTorch:
* Hereda de `torch.nn.Module` - los módulos pueden estar anidados - de hecho, incluso las clases de capa `Conv2d` y `Linear` heredan de `torch.nn.Module`.
* Un modelo tendrá una función `__init__()`, donde instanciará sus capas, y cargará cualquier artefacto de datos que pueda necesitar (por ejemplo, un modelo de PNL podría cargar un vocabulario).
* Un modelo tendrá una función `forward()`. Aquí es donde ocurre el cálculo real: Una entrada se pasa a través de las capas de la red y varias funciones para generar una salida.
* Aparte de eso, puedes construir tu clase modelo como cualquier otra clase de Python, añadiendo cualquier propiedad y método que necesites para soportar el cálculo de tu modelo.

Vamos a instanciar este objeto y ejecutar un ejemplo de entrada a través de él.

In [5]:
net = LeNet()
print(net)                         # what does the object tell us about itself?

input = torch.rand(1, 1, 32, 32)   # stand-in for a 32x32 black & white image
print('\nImage batch shape:')
print(input.shape)

output = net(input)                # we don't call forward() directly
print('\nRaw output:')
print(output)
print(output.shape)

LeNet(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

Image batch shape:
torch.Size([1, 1, 32, 32])
MaxPool conv1 tensor([[[[0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
           0.0000e+00, 0.0000e+00],
          [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
           0.0000e+00, 0.0000e+00],
          [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
           0.0000e+00, 0.0000e+00],
          ...,
          [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
           0.0000e+00, 0.0000e+00],
          [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
           0.0000e+00, 0.0000e+00],
          [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
           0.0000e+00, 0.0000e+00]],

     

Hay algunas cosas importantes que suceden arriba:

Primero, instanciamos la clase `LeNet`, e imprimimos el objeto `net`. Una subclase de `torch.nn.Module` informará de las capas que ha creado y de sus formas y parámetros. Esto puede proporcionar una visión general de un modelo si desea obtener la esencia de su procesamiento.

A continuación, creamos una entrada ficticia que representa una imagen de 32x32 con 1 canal de color. Normalmente, usted cargaría un mosaico de imagen y lo convertiría en un tensor de esta forma.

Los modelos de PyTorch asumen que están trabajando en *lotes* de datos - por ejemplo, un lote de 16 de nuestras imágenes tendría la forma `(16, 1, 32, 32)`. Como sólo vamos a utilizar una imagen, creamos un lote de 1 con la forma `(1, 1, 32, 32)`.

Pedimos al modelo una inferencia llamándolo como una función: `net(input)`. La salida de esta llamada representa la confianza del modelo en que la entrada representa un dígito concreto. (Como esta instancia del modelo no ha aprendido nada todavía, no deberíamos esperar ver ninguna señal en la salida). Si observamos la forma de la "salida", podemos ver que también tiene una dimensión de lote, cuyo tamaño debería coincidir siempre con la dimensión del lote de entrada. Si hubiéramos pasado un lote de entrada de 16 instancias, la "salida" tendría una forma de `(16, 10)`.
